In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
data.to_sql('boston', con, index=False, if_exists='replace')

506

In [5]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [6]:
pd.read_sql(
    '''--sql
    /*
    условие: СТОЛБЕЦ IS NULL и СТОЛБЕЦ = '' выводят везде 0
    */
    SELECT 'CRIM' AS column_name, COUNT(CRIM) AS count_null
    FROM boston
    WHERE CRIM IS NULL
    UNION ALL
    SELECT 'ZN' AS column_name, COUNT(ZN) AS count_null
    FROM boston
    WHERE ZN IS NULL
    UNION ALL
    SELECT 'INDUS' AS column_name, COUNT(INDUS) AS count_null
    FROM boston
    WHERE INDUS IS NULL
    UNION ALL
    SELECT 'CHAS' AS column_name, COUNT(CHAS) AS count_null
    FROM boston
    WHERE CHAS IS NULL
    UNION ALL
    SELECT 'NOX' AS column_name, COUNT(NOX) AS count_null
    FROM boston
    WHERE NOX IS NULL
    ''',
    con,
)

,column_name,count_null
0,CRIM,0
1,ZN,0
2,INDUS,0
3,CHAS,0
4,NOX,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [7]:
pd.read_sql(
    '''--sql
    SELECT 'CRIM' AS column_name, COUNT(DISTINCT CRIM) AS unique_val_cnt
    FROM boston
    UNION ALL
    SELECT 'ZN' AS column_name, COUNT(DISTINCT ZN) AS unique_val_cnt
    FROM boston
    UNION ALL
    SELECT 'INDUS' AS column_name, COUNT(DISTINCT INDUS) AS unique_val_cnt
    FROM boston
    UNION ALL
    SELECT 'CHAS' AS column_name, COUNT(DISTINCT CHAS) AS unique_val_cnt
    FROM boston
    UNION ALL
    SELECT 'NOX' AS column_name, COUNT(DISTINCT NOX) AS unique_val_cnt
    FROM boston
    --limit 5
    ''',
    con,
)

,column_name,unique_val_cnt
0,CRIM,504
1,ZN,26
2,INDUS,76
3,CHAS,2
4,NOX,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [28]:
pd.read_sql(
    '''--sql
    WITH a AS (
    select 'CRIM' AS column_name,min(CRIM) as min
    FROM boston
    union all
    select 'ZN' AS column_name,min(ZN) as min
    FROM boston 
    union all
    select 'INDUS' AS column_name,min(INDUS) as min
    FROM boston
    union all
    select 'CHAS' AS column_name,min(CHAS) as min
    FROM boston
    union all
    select 'NOX' AS column_name,min(NOX) as min
    FROM boston
    )
    ,
    b as
    (select 'CRIM' AS column_name,avg (CRIM) as median from
    (select  CRIM
    FROM boston
    order by CRIM
    limit 1 offset (select count(*) from boston) / 2)
    union all
    select 'ZN' AS column_name,avg (ZN) as median from
    (select  ZN
    FROM boston
    order by ZN
    limit 1 offset (select count(*) from boston) / 2)
    union all
    select 'INDUS' AS column_name,avg (INDUS) as median from
    (select  INDUS
    FROM boston
    order by INDUS
    limit 1 offset (select count(*) from boston) / 2)
    union all
    select 'CHAS' AS column_name,avg (CHAS) as median from
    (select  CHAS
    FROM boston
    order by CHAS
    limit 1 offset (select count(*) from boston) / 2)
    union all
    select 'NOX' AS column_name,avg (NOX) as median from
    (select  NOX
    FROM boston
    order by NOX
    limit 1 offset (select count(*) from boston) / 2))
    select
    a.column_name as column_name
    from a join b 
    on a.column_name = b.column_name
    and a.min=b.median
    
   /* 
    SELECT 'ZN' AS column_name
    , PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY ZN) AS median -- PERCENTILE_CONT не работает
    , MIN(ZN) AS min_value
    FROM boston
    -- UNION ALL
    )
    SELECT * FROM a;*/
    ''',
    con,
)

,column_name
0,ZN
1,CHAS


<i>Вывод: </i>

In [34]:
# Temp space
pd.read_sql(
    '''--sql    
    SELECT 'CRIM' AS column_name
    , CRIM
    , (CRIM * 0.5) AS median
    --, MIN(CRIM) AS min_value

    FROM boston
    --WHERE CRIM * 0.5 = MIN(CRIM)
    ''',
    con,
)

,column_name,CRIM,median
0,CRIM,0.00632,0.003160
1,CRIM,0.02731,0.013655
2,CRIM,0.02729,0.013645
3,CRIM,0.03237,0.016185
4,CRIM,0.06905,0.034525
...,...,...,...
501,CRIM,0.06263,0.031315
502,CRIM,0.04527,0.022635
503,CRIM,0.06076,0.030380
504,CRIM,0.10959,0.054795


4. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [14]:

pd.read_sql(
    '''--sql
    SELECT 
    LSTAT,
    MEDV,
    dense_rank() OVER (PARTITION BY MEDV ORDER BY LSTAT) as rank
    from boston
    where MEDV in
    (SELECT max(MEDV) FROM boston)
    
    ''',
    con,
)

,LSTAT,MEDV,rank
0,1.73,50.0,1
1,1.92,50.0,2
2,2.88,50.0,3
3,2.96,50.0,4
4,2.97,50.0,5
5,3.16,50.0,6
6,3.26,50.0,7
7,3.32,50.0,8
8,3.70,50.0,9
9,3.73,50.0,10


<i>Выыод: Исходя из таблицы, в самых дорогих домах живёт не более 10% с нмзким статусом населения</i>

5. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [10]:

pd.read_sql(
    '''--sql
    SELECT 
    case when CHAS = 1 then 'River' else 'No river' end AS CHAS
    , CHAS
    , round(AVG(MEDV * 1000),2) AS avg_medv
    FROM boston
    GROUP BY CHAS
    limit 5
    ''',
    con,
)

,CHAS,CHAS,avg_medv
0,No river,0.0,22093.84
1,River,1.0,28440.00


6. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [40]:

pd.read_sql(
    '''--sql
    WITH avg AS
    (SELECT 
    CHAS,
    avg(CRIM) AS CRIM,
    avg(ZN) AS ZN,
    avg(INDUS) AS INDUS,
    avg(CHAS) AS CHAS
    FROM boston
    group by CHAS)
    select
    case when then 
    from avg 

    ''',
    con,
)

,CHAS,CRIM,ZN,INDUS,CHAS
0,0.0,3.723215,11.634820,11.019193,0.0
1,1.0,1.851670,7.714286,12.719143,1.0


<i>Вывод: </i>

7. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [7]:

pd.read_sql(
    '''--sql
    SELECT INDUS, NOX
    FROM boston
    group by INDUS, NOX
    ''',
    con,
)

,INDUS,NOX
0,0.46,0.422
1,0.74,0.410
2,1.21,0.401
3,1.22,0.403
4,1.25,0.429
...,...,...
93,19.58,0.605
94,19.58,0.871
95,21.89,0.624
96,25.65,0.581


<i>Вывод: </i>